In [1]:
# Import os and disable tensorflow warnings
import os

import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

In [2]:
# Import libraries
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Flatten, BatchNormalization, Dropout

In [3]:
# Load reviews dataset
reviews_path = 'combined-dataset/final_reviews_data.csv'
data = pd.read_csv(reviews_path)

In [4]:
# Encode types column and Tokenize reviews column
encoder = LabelEncoder()
data['types_encoded'] = encoder.fit_transform(data['types'])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['review'])
sequences = tokenizer.texts_to_sequences(data['review'])

# Pad sequences
max_sequence_length = max(map(len, sequences), default=0)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [5]:
# Create features set and normalize sentiment for label
X = {
    'review': padded_sequences,
    'types': data['types_encoded'].values
}

Y = data['sentiment'].values

In [ ]:
## Define Models Layer
# Input
review_input = Input(shape=(max_sequence_length,), name='review')
types_input = Input(shape=(1,), name='types')

# Embedding and LSTM for review
review_embedding = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(review_input)
review_lstm = LSTM(128)(review_embedding)

# Embedding and Flatten for types
types_embedding = Embedding(input_dim=len(encoder.classes_), output_dim=128)(types_input)
types_flat = Flatten()(types_embedding)

# Concatenate review and types
concatenated = Concatenate()([review_lstm, types_flat])

# Dense layers
dense_1 = Dense(128, activation='relu')(concatenated)
batch_1 = BatchNormalization()(dense_1)
dropout_1 = Dropout(0.2)(batch_1)

dense_2 = Dense(64, activation='relu')(dropout_1)
batch_2 = BatchNormalization()(dense_2)
dropout_2 = Dropout(0.2)(batch_2)

dense_3 = Dense(32, activation='relu')(dropout_2)
batch_3 = BatchNormalization()(dense_3)
dropout_3 = Dropout(0.2)(batch_3)

output = Dense(1, activation='linear')(dropout_3)

In [ ]:
# Create and Compile the Model
model = Model(inputs=[review_input, types_input], outputs=output)
model.summary()
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
model.fit([X['review'], X['types']], Y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
loss, mae = model.evaluate([X['review'], X['types']], Y)
print(f'Model has a loss of {loss} and a mean absolute error of {mae}')

In [ ]:
model.save('model.keras')

In [6]:
content_model = load_model('model.keras')

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

def get_place_recommendations(model, place_id, dataset, top_n=10):
    place_idx = dataset[dataset['id'] == place_id].index[0]
    place_review = X['review'][place_idx]
    place_types = X['types'][place_idx]
    
    predicted_sentiments = model.predict([X['review'], X['types']])
    
    place_vector = np.concatenate([place_review, [place_types]])
    all_vector = np.hstack([X['review'], X['types'].reshape(-1, 1)])
    similarities = cosine_similarity([place_vector], all_vector)[0]
    
    similar_indices = np.argsort(similarities)[-top_n:][::-1]
    similar_places = dataset.iloc[similar_indices]
    
    return similar_places, predicted_sentiments[similar_indices]

In [12]:
place_id = 'ChIJIaGQ-Eg60i0RnT9pzyD_gvM'

recommendations = get_place_recommendations(content_model, place_id, data)
print(recommendations)

1024/1024 [==============================] - 22s 21ms/step
(                                id                                    types  \
2552   ChIJIaGQ-Eg60i0RnT9pzyD_gvM                               cafe, food   
18987  ChIJjT4DJK5G0i0R3pksi46oHZY                               cafe, food   
23697  ChIJ1dMM21FH0i0Ru3XNx9p8_S0  indonesian_restaurant, restaurant, food   
31801  ChIJAe-Pc09F0i0RFm0SEDsDyU8                       tourist_attraction   
13981  ChIJxdi5l84n0i0RyUyMbwuSf1w                         restaurant, food   
20320  ChIJqeoHnm2H0S0Re7c7kU8NUtE                 park, tourist_attraction   
2716   ChIJ65f5180V0i0RkMx79fIo0Ts                         restaurant, food   
4039   ChIJz0XREtZH0i0RowHdlImQUYQ           coffee_shop, cafe, store, food   
30428  ChIJQ7sXNoZB0i0RLRxhrTl5500                           hotel, lodging   
31575  ChIJt_0lSYZz0i0RfM-BdC8kMhU                         restaurant, food   

      review_number                                             review